In [22]:
import osmnx as ox
import matplotlib.pyplot as plt
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import math
import random

from shapely.geometry import box, Point, LineString
from heapq import heappush, heappop
from itertools import count
from networkx.algorithms.flow import edmonds_karp

In [2]:
place_name = "Singapore, Central, Singapore"
graph = ox.save_load.load_graphml(filename="Singapore_drive.graphml", folder=None)

# Stores nodes and edges along with other attributes
type(graph)

networkx.classes.multidigraph.MultiDiGraph

In [3]:
# Preprocess edges for easy computation
nodes, edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)

# Drop unnecessary columns
edges = edges.drop(['lanes','bridge','tunnel','ref','junction','access','service','width'], axis=1)

# Remove null, list, and convert to int ---> maxspeed
edges['maxspeed'].fillna('40', inplace=True)

i = 0
for x in edges['maxspeed']:
    if type(x) == list:
        edges['maxspeed'][i] = max(x)
    i += 1    
    
edges['maxspeed'] = edges['maxspeed'].astype(int)

# Upper bound length
edges['length'] = [math.ceil(x) for x in edges['length']]

# Define capacity of cars on the road
capacity = [random.randrange(835,905) for i in range(len(edges['u']))]
edges['capacity'] = capacity

# Define cost as travel time in sec*100
travel_time = []
for l,s in zip(edges['length'], edges['maxspeed']):
    t = (l*3600)/(s*1000)
    travel_time.append(int(t*100))

edges['travel_time'] = travel_time

/home/orionstar25/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [4]:
edges.head()

,u,v,key,osmid,oneway,name,highway,maxspeed,length,geometry,capacity,travel_time
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60,15,"LINESTRING (103.93262 1.35658, 103.93268 1.35669)",868,90
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60,28,"LINESTRING (103.93262 1.35658, 103.93251 1.356...",846,168
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40,21,"LINESTRING (103.92124 1.31918, 103.92142 1.31922)",878,189
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40,16,"LINESTRING (103.92124 1.31918, 103.92118 1.319...",876,144
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60,14,"LINESTRING (103.93275 1.35665, 103.93269 1.35654)",878,84


In [5]:
G = ox.gdfs_to_graph(nodes, edges)

In [6]:
H = nx.DiGraph(G)
type(H)

networkx.classes.digraph.DiGraph

### Finding shortest route between centroid of map and another point

In [7]:
# Boundary Box of entire area
bbox = box(*edges.unary_union.bounds)
orig_point = bbox.centroid
print(orig_point)

POINT (103.8228094 1.35161845)


In [8]:
target_point = nodes[nodes['osmid'] == 1820000257].geometry.values[0]
print(target_point)

POINT (103.9326163 1.3565805)


In [9]:
orig_xy = (orig_point.y, orig_point.x)
target_xy = (target_point.y, target_point.x)

In [10]:
def find_nearest_nodes(source, dist, G):
    # Create a bounding box around source of min distance in all directions
    (north, south, east, west) = ox.bbox_from_point(point=source, distance=dist)
    
    # Create truncated graph from box
    small_graph = ox.truncate_graph_bbox(G, north, south, east, west)
    
    # Get all nodes inside the bbox --> candidate destinations
    nodes = ox.graph_to_gdfs(small_graph, nodes=True, edges=False)
    candidate_dest = nodes['osmid'].tolist()
    
    return candidate_dest  


In [23]:
# Max distance for walking is 720m (10 min * 1.2 m/s)
# Find all nodes within 720 m from starting point (target_xy) ---> candidate destinations

candidate_dest = find_nearest_nodes(target_xy, 720, graph)

In [12]:
orig_node = ox.get_nearest_node(graph, orig_xy, method='euclidean')
print(orig_node)

5698333646


In [28]:
# Perform ford-fulkerson for all paths
max_flow = 0
target = 0
r = nx.DiGraph()

for candidate_node in candidate_dest:    
    R = edmonds_karp(H, orig_node, candidate_node)    
    if R.graph['flow_value'] > max_flow:
        max_flow = R.graph['flow_value']
        target = candidate_node
        r = R

In [48]:
print(max_flow, target)

837 1820000257


In [42]:
route = nx.shortest_path(r, source=orig_node, target=target, weight='travel_time')
print(route)

[5698333646, 5698333649, 3784797715, 3784797714, 3998723969, 1318459258, 1318459222, 5698393564, 1318459289, 1318459219, 4727949660, 6913094587, 206256152, 5648108236, 5952941494, 5984801300, 6895851975, 1318498426, 248889863, 206256177, 1318498406, 1857011360, 5698520999, 5698521003, 3251145329, 206089468, 206256184, 175280151, 439978076, 1794586972, 239934510, 239934388, 924800650, 439991742, 439991732, 239905692, 924799021, 239905578, 239886387, 239886200, 239885744, 633167565, 1561754904, 395271696, 244993715, 244993663, 244993823, 1790030776, 1831390121, 1819975128, 1819975126, 1819975122, 4668456813, 1820026224, 1820026216, 4662056865, 1820000257]
